## setup

In [ ]:
TARGET = "Transported"
TRAIN_PATH = "titanic/train.csv"
TEST_PATH = "titanic/test.csv"
SUBMIT_NAME = "submission.csv"
ID_COL = "Id"
TASK = "regression"

## dependencies

In [ ]:
%pip install catboost --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## catboost:

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

target_col = TARGET
id_col = ID_COL
feature_cols = [c for c in train_df.columns if c not in [target_col, id_col]]

categorical_cols = [c for c in feature_cols if train_df[c].dtype == 'object' or train_df[c].dtype.name == 'category']

for col in categorical_cols:
    train_df[col] = train_df[col].astype(str).fillna('nan')
    test_df[col] = test_df[col].astype(str).fillna('nan')

X_train, X_valid, y_train, y_valid = train_test_split(
    train_df[feature_cols], train_df[target_col], test_size=0.2, random_state=42
)

model_type = TASK

if model_type == "regression":
    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        eval_metric='RMSE',
        random_seed=42,
        task_type="GPU",
        verbose=100,
        use_best_model=True
    )
else:
    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        eval_metric='Accuracy',
        random_seed=42,
        task_type="GPU",
        verbose=100,
        use_best_model=True
    )

train_pool = Pool(X_train, y_train, cat_features=categorical_cols)
valid_pool = Pool(X_valid, y_valid, cat_features=categorical_cols)

model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=50)

y_pred = model.predict(X_valid)

test_pred = model.predict(test_df[feature_cols])

submission = pd.DataFrame({
    id_col: test_df[id_col],
    target_col: test_pred
})

submission.to_csv(SUBMIT_NAME, index=False)
print(f"Submission saved to {SUBMIT_NAME}")

data loaded
made features
starting training
